In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf

# Import our input dataset
tele_df = pd.read_csv('bank_telemarketing.csv')
tele_df.head()

2022-11-02 17:43:44.741914: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


,Age,Job,Marital_Status,Education,Default_Credit,Housing_Loan,Personal_Loan,Subscribed
0,56,other,married,Primary_Education,no,no,no,no
1,37,services,married,Secondary_Education,no,yes,no,no
2,40,admin,married,Primary_Education,no,no,no,no
3,56,services,married,Secondary_Education,no,no,yes,no
4,59,admin,married,Professional_Education,no,no,no,no


**В отличие от нейронных сетей и моделей глубокого обучения, SVM могут обрабатывать необработанные и обработанные табличные данные. Несмотря на это, мы предварительно обработаем набор данных и будем использовать предварительно обработанные данные для обучения обеих моделей - это обеспечивает справедливое сравнение. Для нашего первого рабочего процесса предварительной обработки давайте закодируем наши категориальные переменные, используя класс Scikit-Learn'sOneHotEncoder.**


**Во-первых, мы должны убедиться, что ни одна из наших категориальных переменных не требует ведра. Чтобы проверить это, давайте получим имена столбцов категориальных переменных и проверим их количество уникальных значений. Добавьте и запустите следующий код в блокнот:**

In [3]:
# Generate our categorical variable list
tele_cat = tele_df.dtypes[tele_df.dtypes == "object"].index.tolist()


# Check the number of unique values in each column
tele_df[tele_cat].nunique()

Job               9
Marital_Status    3
Education         4
Default_Credit    2
Housing_Loan      2
Personal_Loan     2
Subscribed        2
dtype: int64

**Глядя на количество уникальных значений для каждой категориальной переменной, не было категорий, требующих сегментирования перед кодированием. Поэтому мы готовы кодировать, добавляя и запуская следующий код в наши ноутбуки:**

In [4]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(tele_df[tele_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(tele_cat)
encode_df.head()

/Users/dang/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Job_admin,Job_blue-collar,Job_entrepreneur,Job_management,Job_other,Job_retired,Job_self-employed,Job_services,Job_technician,Marital_Status_divorced,...,Education_Secondary_Education,Education_Tertiary_Education,Default_Credit_no,Default_Credit_yes,Housing_Loan_no,Housing_Loan_yes,Personal_Loan_no,Personal_Loan_yes,Subscribed_no,Subscribed_yes
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


**Как только у нас появятся закодированные категориальные переменные, нам нужно объединить наши закодированные столбцы обратно в наш исходный DataFrame (а также удалить незакодированные столбцы). Чтобы заменить незакодированные категориальные переменные закодированными переменными, добавьте и запустите следующий код в блокнот:**

In [5]:
# Merge one-hot encoded features and drop the originals
tele_df = tele_df.merge(encode_df,left_index=True, right_index=True)
tele_df = tele_df.drop(tele_cat,1)
tele_df.head()

/Users/dang/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,Age,Job_admin,Job_blue-collar,Job_entrepreneur,Job_management,Job_other,Job_retired,Job_self-employed,Job_services,Job_technician,...,Education_Secondary_Education,Education_Tertiary_Education,Default_Credit_no,Default_Credit_yes,Housing_Loan_no,Housing_Loan_yes,Personal_Loan_no,Personal_Loan_yes,Subscribed_no,Subscribed_yes
0,56,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,40,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,59,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


**Теперь мы должны разделить наши данные на наборы для обучения и тестирования перед стандартизацией, чтобы не включать значения тестирования в шкалу - значения тестирования предназначены только для оценки. Чтобы выполнить разделение обучения/тестирования и стандартизировать наши числовые переменные, добавьте и запустите следующий код в блокноте**

In [6]:
# Remove loan status target from features data
y = tele_df.Subscribed_yes.values
X = tele_df.drop(columns=["Subscribed_no","Subscribed_yes"]).values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

**После стандартизации переменных как в данных обучения, так и в тестировании наш набор данных готов для обеих моделей. Во-первых, мы обучим и оценим наш SVM.**

**SVMs can be built using Scikit-learn's SVCclass in the svmmodule.**

**Для наших целей мы будем использовать linear ядро SVM, чтобы попытаться создать линейную границу между группами "Subscribed_yes" и "Subscribed_no". Чтобы создать нашу модель SVM и протестировать производительность, добавьте и запустите следующий код:**

In [7]:
# Create the SVM model
svm = SVC(kernel='linear')

In [8]:
# Train the model
svm.fit(X_train, y_train)

SVC(kernel='linear')

In [9]:
# Evaluate the model
y_pred = svm.predict(X_test_scaled)
print(f" SVM model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 SVM model accuracy: 0.873


**Глядя на выход нашей модели SVM, модель смогла правильно предсказать клиентов, которые подписались примерно в 87% случаев, что является респектабельной моделью первого прохода. Затем нам нужно составить и оценить нашу модель глубокого обучения. Опять же, мы будем использовать наши типичные параметры двоичного классификатора:**
- Our first hidden layer will have an input_dim equal to the length of the scaled feature data X , 10 neuron units, and will use the relu activation function.
- Наш второй скрытый слой будет иметь 5 нейронных units, а также будет использовать функцию активации relu.
- Функция потерь должна быть binary_crossentropy, используя оптимизатор adam.


**В отличие от нашей базовой модели нейронных сетей, мы не хотим использовать в два-три раза больше нейронов в качестве входных переменных - мы не хотим, чтобы наши более глубокие слои перенаправляли входные данные.**

**Чтобы построить и скомпилировать нашу модель глубокого обучения, мы должны добавить и запустить следующий код:**

In [10]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

2022-11-02 17:49:27.795857: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


**Наконец, нам нужно обучить и оценить нашу модель глубокого обучения. Поскольку этот набор данных содержит меньше функций, чем другие наборы данных, которые мы использовали ранее, нам нужно обучаться максимум в течение 50 эпох. Опять же, мы должны добавить и запустить следующий код:**

In [11]:
# Train the model 
fit_model = nn.fit(X_train_scaled, y_train, epochs=50) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
715/715 [==============================] - 2s 2ms/step - loss: 0.6128 - accuracy: 0.7873
Epoch 2/50
715/715 [==============================] - 1s 2ms/step - loss: 0.4678 - accuracy: 0.8734
Epoch 3/50
715/715 [==============================] - 1s 2ms/step - loss: 0.4114 - accuracy: 0.8734
Epoch 4/50
715/715 [==============================] - 1s 2ms/step - loss: 0.3877 - accuracy: 0.8735
Epoch 5/50
715/715 [==============================] - 1s 2ms/step - loss: 0.3767 - accuracy: 0.8734
Epoch 6/50
715/715 [==============================] - 1s 2ms/step - loss: 0.3732 - accuracy: 0.8732
Epoch 7/50
715/715 [==============================] - 1s 2ms/step - loss: 0.3721 - accuracy: 0.8734
Epoch 8/50
715/715 [==============================] - 1s 2ms/step - loss: 0.3716 - accuracy: 0.8735
Epoch 9/50
715/715 [==============================] - 1s 2ms/step - loss: 0.3714 - accuracy: 0.8737
Epoch 10/50
715/715 [==============================] - 1s 2ms/step - loss: 0.3712 - accuracy: 0.8736

**Глядя на результаты нашего сравнительного анализа, модели SVM и глубокого обучения достигли точности прогнозирования около 87%. Кроме того, обе модели занимают одинаковое количество времени для обучения входным данным. Единственным заметным различием между двумя моделями является реализация - объем кода, необходимый для сборки и обучения SVM, значительно меньше, чем сопоставимая модель глубокого обучения. В результате многие специалисты по анализу данных предпочтут использовать SVM по умолчанию, а затем при необходимости обращаться к моделям глубокого обучения.**